In [1]:
import os

In [2]:
%pwd

'/Users/zeglazi/Desktop/End-To-End-ML-Project/Chicken-Disease-Classification-Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/zeglazi/Desktop/End-To-End-ML-Project/Chicken-Disease-Classification-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_file : Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file = config.local_data_file,
            unzip_file= config.unzip_file
        )
        
        return data_ingestion_config
    
          

In [8]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_dir))}")
            
            
    def extract_zip_file(self):
        
        unzip_path = self.config.unzip_file
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
        

In [10]:
try :
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-24 19:48:33,033: INFO: common: yaml file : config/config.yaml loaded successfully]
[2024-05-24 19:48:33,037: INFO: common: yaml file : params.yaml loaded successfully]
[2024-05-24 19:48:33,039: INFO: common: Created directory at : artifacts]
[2024-05-24 19:48:33,040: INFO: common: Created directory at : artifacts/data_ingestion]
[2024-05-24 19:48:36,237: INFO: 2659251300: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D510:373683:D513C5:DF7F5B:6650D2F1
Accept-Ranges: bytes
Date: Fri, 24 May 2024 17:48:34 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs10564-MRS
X-Ca